# Algoritm inteligent + small data

### 1. Descriere EDA a datelor:
- Continut:<br>
Dataset-ul contine **4920** de exemple, unde fiecare input are **17** atribute (*Simptom_1, Simptom_2, ..., Simpotom_17*)<br>
Dataset-ul are ca scop corelarea unor simptome cu un **diagnostic** (variabila tinta). Atributele sunt de tip string, reprezentand un simptom asociat diagnosticului din linia respectiva

- Analiza statistica:<br>
Dataset-ul cuprinde 41 de diagnostice unice si 131 de simptome distincte. Dataset-ul este echilibrat, avand pentru fiecare diagnostic 120 de exemple.<br>
Analizand mai atent setul de date, am observat ca multe dintre perechile (colectie_simptome - diagnostic) sunt duplicate. Daca nu modific in niciun fel setul de date, atunci exista riscul ca procesul de testare al modelului dupa antrenare sa fie sabotat - exista sansa ca multe dintre exemplele din train set sa apara si in test set (**data leakage**), ceea ce ar duce la o valoare a acuratetii irelevanta (am testat pastrand duplicatele si am obtinut o acuratete de **100%**, evident nerealista)

In [7]:
import pandas as pd

df = pd.read_csv('data/DiseaseAndSymptoms.csv')

# Transformarea dataset-ului in perechi de tipul (lista_simptome - diagnostic)

symptom_cols = [col for col in df.columns if col.startswith("Symptom_")]
df["lista_simptome"] = df[symptom_cols].apply(lambda x: [s.strip() for s in x.dropna().astype(str) if s.strip() != ''], axis=1)
df.drop(columns=symptom_cols, inplace=True)

print(df.head())
print()

# determinare simptome unice

all_symptoms = [s for lista in df['lista_simptome'] for s in lista if isinstance(lista, list)]
unique_symptoms = set(all_symptoms)

print(f'{len(unique_symptoms)} unique symptoms')

            Disease                                     lista_simptome
0  Fungal infection  [itching, skin_rash, nodal_skin_eruptions, dis...
1  Fungal infection  [skin_rash, nodal_skin_eruptions, dischromic _...
2  Fungal infection  [itching, nodal_skin_eruptions, dischromic _pa...
3  Fungal infection          [itching, skin_rash, dischromic _patches]
4  Fungal infection         [itching, skin_rash, nodal_skin_eruptions]

131 unique symptoms


#### Preprocesare date:
- voi elimina toate liniile duplicate (pentru a masura capacitatea de generalizare a modelului);
- pentru fiecare diagnostic, voi asocia un vector binar care sa ilustreze corelatia dintre un diagnostic, respectiv un simptom (1, daca simptomul este prezent, 0 altfel)
- voi codifica fiecare diagnostic cu un intreg

In [11]:
from sklearn.preprocessing import MultiLabelBinarizer

# eliminare duplicate

df['tuple_simptome'] = df['lista_simptome'].apply(lambda x: tuple(sorted(x)))
df = df.drop_duplicates(subset=['Disease', 'tuple_simptome']).reset_index(drop=True)
df = df.drop(columns=['tuple_simptome'])

print(df.shape)
print(df.head())

# one-hot encoding pentru simptome

mlb = MultiLabelBinarizer()
X = mlb.fit_transform(df['lista_simptome'])
df_aux = pd.DataFrame(X, columns=mlb.classes_)
df_aux['Disease'] = df['Disease'].values
df = df_aux

print(df.head())
print()

(304, 2)
            Disease                                     lista_simptome
0  Fungal infection  [itching, skin_rash, nodal_skin_eruptions, dis...
1  Fungal infection  [skin_rash, nodal_skin_eruptions, dischromic _...
2  Fungal infection  [itching, nodal_skin_eruptions, dischromic _pa...
3  Fungal infection          [itching, skin_rash, dischromic _patches]
4  Fungal infection         [itching, skin_rash, nodal_skin_eruptions]
   abdominal_pain  abnormal_menstruation  acidity  acute_liver_failure  \
0               0                      0        0                    0   
1               0                      0        0                    0   
2               0                      0        0                    0   
3               0                      0        0                    0   
4               0                      0        0                    0   

   altered_sensorium  anxiety  back_pain  belly_pain  blackheads  \
0                  0        0          0           0

### 2. Antrenare folosind algoritm inteligent

Pentru rezolvarea problemei de clasificare, voi folosi libraria **XGBoost**.
XGBoost se bazeaza pe **gradient boosting algorithm**.

#### Cum functioneaza XGBoost?
1. Incepe prin antrenarea unui arbore de decizie pe setul de antrenament.
2. Dupa evaluarea erorii obtinute de primul arbore de decizie, construieste un nou arbore care va fi antrenat pe baza erorilor facute de primul, scopul celui de-al dolea fiind sa corecteze erorile facute de primul.
3. Acest proces se repeta, fiecare nou arbore de decizie incercand sa corecteze greselile celor de dinaintea lui.
4. Predictia finala este reprezentata de suma deciziilor tuturor arborilor.

#### Avantajele utilizarii XGBoost:
1. XGBoost include diferite tehnici pentru **prevenirea overfitting-ului** (ex: utilizarea regularizarii, prioritizand astfel simplificarea arborilor complecsi)
2. Performanta ridicata prin optimizarea utilizarii memoriei prioritizand utilizarea cache-ului de la CPU
3. Suporta foarte bine feature-uri binare (cum e in cazul nostru)

#### Descriere metodologie:
1. In urma analizei setului de date, am hotarat sa il impart doar in **train set / test set (80% / 20%)**, deoarece este un dataset de dimensiuni mici.
2. Pentru o evaluare cat mai realista, voi folosi **cross-validation**, iar ca metrici: **accuracy**, **precision_macro**, **recall_macro**.
3. Hiperparametri folositi vor fi:
- **n_estimators**: numarul de arbori pe care algoritmul ii va construi
- **learning_rate**: viteza cu care converge modelul spre parametri optimi
- **max_depth**: adancimea maxima a arborilor de decizie
- **subsample**: proporitia de linii pe care va fi antrenat fiecare arbore
- **colsample_bytree**: proportia de coloane folosite pentru fiecare arbore

In [20]:
import xgboost as xgb
from sklearn.model_selection import cross_validate, StratifiedKFold
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
y_encoded = le.fit_transform(df['Disease'])

model = xgb.XGBClassifier(
    objective='multi:softmax',
    num_class=len(mlb.classes_),
    n_estimators=100,
    learning_rate=0.02,
    max_depth=3,
    subsample=0.8,
    colsample_bytree=0.8,
    random_state=42,
)

kfold = StratifiedKFold(n_splits=5, shuffle=True, random_state=42)
scores = cross_validate(
    model, X, y_encoded, cv=kfold,
    scoring=['accuracy', 'precision_macro', 'recall_macro', 'f1_macro'],
)

print(f'Accuracy: {scores["test_accuracy"].mean()*100:.3f}%')
print(f'Precision: {scores["test_precision_macro"].mean()*100:.3f}%')
print(f'Recall: {scores["test_recall_macro"].mean()*100:.3f}%')
print(f'F1: {scores["test_f1_macro"].mean()*100:.3f}%')

/Users/stiubedenis/PyCharmMiscProject/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/stiubedenis/PyCharmMiscProject/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Users/stiubedenis/PyCharmMiscProject/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, mo

Accuracy: 85.842%
Precision: 79.268%
Recall: 82.683%
F1: 79.515%


/Users/stiubedenis/PyCharmMiscProject/.venv/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


In [24]:
# afisare exemple prezise corect, respectiv exemple prezise gresit

from sklearn.model_selection import cross_val_predict

y_prec_cv = cross_val_predict(model, X, y_encoded, cv=kfold)

y_pred_labels = le.inverse_transform(y_prec_cv)
y_true_labels = le.inverse_transform(y_encoded)

correct = [(true, pred, i) for i, (true, pred) in enumerate(zip(y_true_labels, y_pred_labels)) if true == pred]
wrong   = [(true, pred, i) for i, (true, pred) in enumerate(zip(y_true_labels, y_pred_labels)) if true != pred]

for c in correct[:5]:
    print(f'Index {c[2]}: True - {c[0]}, Predicted - {c[1]}')
    print()
for w in wrong[:5]:
    print(f'Index {w[2]}: True - {w[0]}, Predicted - {w[1]}')
    print()

Index 5: True - Allergy, Predicted - Allergy

Index 6: True - Allergy, Predicted - Allergy

Index 7: True - Allergy, Predicted - Allergy

Index 9: True - Allergy, Predicted - Allergy

Index 10: True - GERD, Predicted - GERD

Index 0: True - Fungal infection, Predicted - Drug Reaction

Index 1: True - Fungal infection, Predicted - Acne

Index 2: True - Fungal infection, Predicted - Drug Reaction

Index 3: True - Fungal infection, Predicted - Drug Reaction

Index 4: True - Fungal infection, Predicted - Drug Reaction



#### Analiza rezultate:
In urma incercarii mai multor valori pentru anumiti hiperparametrii (n_estimators = 100..300, learning_rate = 0.01..0.1), consider ca am reusit sa ajung la un model cu o performanta buna si stabila.
Valorile pentru metricile folosite sunt apropiate ca valoare (aprox 81% - ceea ce indica stabilitate). Tinand cont de faptul ca sunt 41 de clase, deci prin prezicere aleatorie sansele de succes sunt de aprox 2.4%, o acuratete de 85% este foarte buna.